In [9]:
# Melakukan import dataset standard iris dan memisahkan data dan target
from sklearn import datasets
iris_dataset = datasets.load_iris()
X_iris = iris_dataset.data
Y_iris = iris_dataset.target
print(X_iris[0:5])
print(Y_iris[0:5])

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]]
[0 0 0 0 0]


In [11]:
# Melakukan import dataset csv play tennis dan memisahkan data dan target
import pandas
ori_data_frame = pandas.read_csv('tennis.csv')

# axis = 1 artinya mau drop column
data_frame = ori_data_frame.drop('play', axis=1)
data_frame = data_frame.select_dtypes(include=[object])
# print(data_frame.columns)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_2 = data_frame.apply(le.fit_transform)
# print(df_2)

one_hot_encoder = preprocessing.OneHotEncoder()
one_hot_encoder.fit(df_2)
one_hot_labels = one_hot_encoder.transform(df_2).toarray()

X_tennis = one_hot_labels
Y_tennis = []

for row in ori_data_frame.index:
    Y_tennis.append(ori_data_frame['play'][row])

print(X_tennis[0:5])
print(Y_tennis[0:5])

     outlook  temp humidity
0      sunny   hot     high
1      sunny   hot     high
2   overcast   hot     high
3      rainy  mild     high
4      rainy  cool   normal
5      rainy  cool   normal
6   overcast  cool   normal
7      sunny  mild     high
8      sunny  cool   normal
9      rainy  mild   normal
10     sunny  mild   normal
11  overcast  mild     high
12  overcast   hot   normal
13     rainy  mild     high
[[ 0.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  1.  0.  0.  0.  1.]]
['no', 'no', 'yes', 'yes', 'yes']


In [3]:
# Melakukan pembelajaran full training dengan DecisionTree
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_iris, Y_iris)
clf_tree_2 = tree.DecisionTreeClassifier()
clf_tree_2 = clf_tree_2.fit(X_tennis, Y_tennis)
# Menampilkan hasil prediksi terhadap 5 instance awal
# print(clf_tree.predict(X_iris[:5]))
# print(clf_tree_2.predict(X_tennis[:5]))

In [6]:
# Menampilkan model pembelajaran tree
dot_model = tree.export_graphviz(clf_tree, out_file = None) 
dot_model_2 = tree.export_graphviz(clf_tree_2, out_file = None) 

import graphviz
import os
os.environ["PATH"] += os.pathsep + 'D:/Graphviz2.38/bin/'
tree_viz = graphviz.Source(dot_model) 
tree_viz.render('tree_iris')
tree_viz_2 = graphviz.Source(dot_model) 
tree_viz_2.render('tree_tennis')

'tree_tennis.pdf'

In [ ]:
# Melakukan pembelajaran full training dengan MLP
from sklearn.neural_network import MLPClassifier
clf_mlp = MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(5, 5), random_state=1)
clf_mlp.fit(X_iris, Y_iris)
# Menampilkan hasil prediksi terhadap 5 instance awal
# print(clf_mlp.predict(X_iris[:5]))

In [ ]:
# Menampilkan model pembelajaran MLP

In [ ]:
# Melakukan import dan export model
from sklearn.externals import joblib
def save_model(model, filename):
    joblib.dump(model, filename)
    
def load_model(filename):
    clf = joblib.load(filename)
    return clf

In [ ]:
# Meng-export full training MLP dan DTree
save_model(clf_tree, "full_training_Tree_Iris.model")
save_model(clf_mlp, "full_training_MLP_Iris.model")

# Me-load full training MLP dan DTree
clf_mlp = load_model("full_training_MLP_Iris.model")
clf_tree = load_model("full_training_Tree_Iris.model")
print(clf_mlp.predict(X_iris[:5]))
print(clf_tree.predict(X_iris[:5]))